<a href="https://colab.research.google.com/github/MahdiTheGreat/Intro-to-language-modeling/blob/main/Assignment_3_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Step 0: Imports

In [ ]:
from tqdm import tqdm
from sklearn.datasets import fetch_20newsgroups # We use the 20 news groups text dataset
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from collections import Counter
import numpy as np
import pandas as pd
import re
import pickle

# Step 1: Fetching data and preprocessing

In [ ]:
newsgroups_train = fetch_20newsgroups(subset='train')

In [ ]:
print(len(newsgroups_train.target_names))

20


In [ ]:
# Split into smaller training sets in percentage
percentage = 0.8
split_index = int(len(newsgroups_train.data) * percentage)
train_data_small = newsgroups_train.data[:split_index]
train_targets_small = newsgroups_train.target[:split_index]

In [ ]:
print(train_data_small[0])

From: lerxst@wam.umd.edu (where's my thing)
Subject: WHAT car is this!?
Nntp-Posting-Host: rac3.wam.umd.edu
Organization: University of Maryland, College Park
Lines: 15

 I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a model name, engine specs, years
of production, where this car is made, history, or whatever info you
have on this funky looking car, please e-mail.

Thanks,
- IL
   ---- brought to you by your neighborhood Lerxst ----







In [ ]:
def extract_body(text):
    # Extract only words using regex
    words = re.findall(r'\b[a-zA-Z]+\b', text)

    # Join the words with spaces (optional)
    cleaned_text = " ".join(words)

    return cleaned_text

removed_headers = extract_body(train_data_small[0])
print(removed_headers)

From lerxst wam umd edu where s my thing Subject WHAT car is this Nntp Posting Host wam umd edu Organization University of Maryland College Park Lines I was wondering if anyone out there could enlighten me on this car I saw the other day It was a door sports car looked to be from the late early It was called a Bricklin The doors were really small In addition the front bumper was separate from the rest of the body This is all I know If anyone can tellme a model name engine specs years of production where this car is made history or whatever info you have on this funky looking car please e mail Thanks IL brought to you by your neighborhood Lerxst


In [ ]:
nltk.download('stopwords')
nltk.download('punkt_tab')

stop_words = set(stopwords.words('english'))

# Initialize structures for the preprocessed corpus
filtered_train = [[] for _ in range(len(train_data_small))]  # Preprocessed articles
flattened_train = []  # A single list of all words in the corpus

# Tokenizing and removing stopwords
print("Processing Articles")
for i, article in tqdm(enumerate(train_data_small), total=len(train_data_small)):
    article_body = extract_body(article) # Only use body and remove headers and footers
    word_tokens = word_tokenize(article_body)  # Tokenize article
    # Remove stop words and add to both filtered_train and flattened_train
    filtered_words = [w.lower() for w in word_tokens if w.lower() not in stop_words]
    filtered_train[i] = filtered_words
    flattened_train.extend(filtered_words)

# Create a vocabulary mapping
unique_words = sorted(set(flattened_train))  # Get unique words
word_to_id = {word: idx for idx, word in enumerate(unique_words)}  # Map word to ID
id_to_word = {idx: word for word, idx in word_to_id.items()}  # Reverse mapping

# Map the filtered articles to integer IDs
int_corpus = [[word_to_id[word] for word in article] for article in filtered_train]

# Display mappings and a small example
print(f"Total unique words: {len(unique_words)}")
print("Word-to-ID mapping example:", {k: word_to_id[k] for k in list(word_to_id)[:5]})
print("Integer Corpus example (first article):", int_corpus[0][:10])

Processing Articles

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ANv\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\ANv\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


100%|██████████| 9051/9051 [00:05<00:00, 1591.41it/s]


Total unique words: 74122
Word-to-ID mapping example: {'aa': 0, 'aaa': 1, 'aaaa': 2, 'aaaaaaaaaaaa': 3, 'aaaaagggghhhh': 4}
Integer Corpus example (first article): [36478, 70000, 66878, 18878, 64337, 61863, 8966, 44346, 49795, 28837]


In [ ]:
word_count = Counter(flattened_train)

In [ ]:
# Extract low-frequency words (occurrence <= 10) into a set
low_frequency_words = {word for word, count in word_count.items() if count <= 10}

In [ ]:
# Filter articles efficiently using set operations
corpus_hf = []
for article in tqdm(int_corpus, desc="Removing LF words"):
    article_set = set(article)
    filtered_article = list(article_set - low_frequency_words)
    corpus_hf.append(filtered_article)

Removing LF words: 100%|██████████| 9051/9051 [00:00<00:00, 81097.88it/s]


In [ ]:
flattened_train = [word for word in flattened_train if word not in low_frequency_words]
voc_size = len(sorted(set(flattened_train)))
print("Number of words in corpus", len(flattened_train))
print("Vocabulary size after removing LF words", voc_size)
print(len(list(set(word for doc in corpus_hf for word in doc))))

Number of words in corpus 1467687
Vocabulary size after removing LF words 14426
74122


# Step 2: Gibbs sampling

In [ ]:
from collections import defaultdict

def lda_gibbs_sampling(corpus, K, alpha, beta, iterations):
    """
    Implements Collapsed Gibbs Sampling for LDA.

    :param corpus: List of lists, where each inner list contains word IDs in a document.
    :param K: Number of topics.
    :param alpha: Dirichlet prior for document-topic distribution.
    :param beta: Dirichlet prior for topic-word distribution.
    :param iterations: Number of Gibbs sampling iterations.
    :return: Topic assignments, document-topic counts, topic-word counts, topic totals.
    """
    # Initialize variables
    D = len(corpus)  # Number of documents
    V = len(list(set(word for doc in corpus for word in doc)))

    doc_word_matrix = np.zeros((D, V), dtype=bool)

    # Count matrices
    ndk = np.zeros((D, K))  # Document-topic counts
    nkw = np.zeros((K, V))  # Topic-word counts
    nk = np.zeros(K)        # Total words in each topic

    # Topic assignments for each word
    z = []  # Topic assignment for each word in corpus
    for d, doc in enumerate(corpus):
        doc_topics = []
        for word in doc:
            topic = np.random.randint(K)  # Randomly assign a topic
            doc_topics.append(topic)
            ndk[d, topic] += 1
            nkw[topic, word] += 1
            nk[topic] += 1
            doc_word_matrix[d, word] = True
        z.append(doc_topics)

    # Gibbs sampling
    for _ in tqdm(range(iterations)):
        for d, doc in enumerate(corpus):
            for i, word in enumerate(doc):
                current_topic = z[d][i]

                # Decrement counts
                ndk[d, current_topic] -= 1
                nkw[current_topic, word] -= 1
                nk[current_topic] -= 1

                # Compute topic probabilities (Maybe do a for loop here instead)
                topic_probs = (ndk[d] + alpha) * (nkw[:, word] + beta) / (nk + beta * V)
                topic_probs /= np.sum(topic_probs)  # Normalize

                # Sample new topic
                new_topic = np.random.choice(K, p=topic_probs)
                z[d][i] = new_topic

                # Increment counts
                ndk[d, new_topic] += 1
                nkw[new_topic, word] += 1
                nk[new_topic] += 1

    return z, ndk, nkw, nk, doc_word_matrix

In [ ]:
# Initialize data
corpus = corpus_hf.copy()
targets = train_targets_small.copy()
topics = newsgroups_train.target_names.copy()

# First parameter combo
z, ndk, nkw, nk, doc_word_matrix = lda_gibbs_sampling(corpus, alpha = 0.1, beta = 0.1, K=5, iterations=150)

# Save output as pickle file
with open('output.pkl', 'wb') as handle:
    pickle.dump((z, ndk, nkw, nk, doc_word_matrix), handle, protocol=pickle.HIGHEST_PROTOCOL)

100%|██████████| 150/150 [52:26<00:00, 20.98s/it]


In [ ]:
# Load output from pickle
with open('output.pkl', 'rb') as handle:
    z, ndk, nkw, nk, doc_word_matrix = pickle.load(handle)

print(len(z), ndk.shape, nkw.shape, nk.shape, doc_word_matrix.shape)

9051 (9051, 5) (5, 74122) (5,) (9051, 74122)


In [ ]:
import pandas as pd
def get_top_words(nkw, id_to_word, top_n=20, method="raw", beta=0.1):
    """
    Get the top words for each topic.

    :param nkw: Topic-word counts (K x V matrix).
    :param id_to_word: Dictionary mapping word IDs to their original words.
    :param top_n: Number of top words to retrieve per topic.
    :param method: "raw" for raw counts, "relative" for relative frequencies.
    :param beta: Dirichlet prior for smoothing (used in relative frequency).
    :return: Dictionary of top words for each topic.
    """
    K, V = nkw.shape
    top_words_per_topic = {}

    if method == "raw":
        # Use raw counts
        for k in range(K):
            top_word_indices = np.argsort(nkw[k, :])[::-1][:top_n]  # Top N words by count
            top_words_per_topic[k] = [idx for idx in top_word_indices]

    elif method == "relative":
        # Compute relative frequencies
        word_totals = np.sum(nkw, axis=0)  # Total count of each word across all topics
        for k in range(K): # for k in topics
            relative_freqs = (nkw[k, :] + beta) / (word_totals + beta * K)  # Smoothed relative frequency
            top_word_indices = np.argsort(relative_freqs)[::-1][:top_n]  # Top N words by relative frequency
            top_words_per_topic[k] = [
                idx for idx in top_word_indices
            ]

    return top_words_per_topic


def top_words_to_df(top_words):
    """
    Display the top words for each topic in a table format.

    :param top_words_per_topic: Dictionary of top words for each topic.
    :param method: Description of the method used ("raw" or "relative").
    """

    top_words_per_topic = {t: [id_to_word[value] for value in values] for t, values in top_words.items()}
    df_top_words = pd.DataFrame.from_dict(top_words_per_topic)
    df_top_words.columns = [f"Topic {i+1}" for i in range(df_top_words.shape[1])]
    return df_top_words

In [ ]:
top_words_per_topic = get_top_words(nkw, id_to_word, top_n=20, method="relative")
df_top_words = top_words_to_df(top_words_per_topic)
df_top_words

,Topic 1,Topic 2,Topic 3,Topic 4,Topic 5
0,encryption,israeli,ftp,baseball,bike
1,secure,turkish,ram,season,bmw
2,nsa,armenia,shipping,hockey,riding
3,escrow,arab,scsi,nhl,orbit
4,crypto,turks,floppy,atheists,honda
5,amendment,arabs,vga,athos,bikes
6,announcement,argic,motif,detroit,motorcycle
7,committed,extermination,upgrade,playoffs,geb
8,enforcement,israelis,cpu,stanley,shuttle
9,armed,zuma,ide,atheism,ama


In [ ]:
print((doc_word_matrix[:,0] & doc_word_matrix[:,1]).sum())

7


In [ ]:
def umass(corpus, n_topics, top_words, top_n, epsilon=1, print_=False):

        coherence = []
        common_words = list(top_words.values())

        for k in range(n_topics):
            # for each topic


            for vi in range(top_n-1):
                # for each word in each topic
                D_vi = 0


                for doc in corpus:
                    # for each document

                    D_vi_vj = 0
                    vj = vi+1

                    if common_words[k][vi] in doc:
                        # word is in the document
                        D_vi = D_vi + 1

                        if common_words[k][vj] in doc:
                            # only check lag-1 words if word vi in doc
                            D_vi_vj = D_vi_vj + 1

                if D_vi != 0:
                    # catch errors
                    coherence.append(np.log10((D_vi_vj+epsilon)/D_vi))

        model_coherence = np.sum(coherence)/n_topics

        if print_: print("coherence for each topic: ", model_coherence)

        return model_coherence

In [ ]:
print(umass(corpus, 5, top_words_per_topic, top_n=20, print_=True))

coherence for each topic:  -37.59953130389676
-37.59953130389676
